<p style="text-align: center; font-size: 30px; color:CadetBlue;">MATHEMATICAL MODELLING IN MACHINE LEARNING - 30554</p>
<p style= "text-align: center; font-size: 18px;">Luca Raffo</p>


### <span style="color:CadetBlue">Abstract</span>
In this upcoming notebook, we're gearing up to dive into the process of training two different classifiers.
The dataset we will work on comprises 1000 samples, each endowed with a collection of 35 features. Each sample has a label, an integer ranging between 0 and 3 (extremes included).  
In particular, we aim to embark on a comparative study of the performances of two renowned algorithms: k-NearestNeighbors and Random Forest.  
We will optimize the hyperparameters for both of these models, in order to exploit their full potential.  
In the end we will conclusively determine which classifier exhibited a better performance on this specific dataset.

### <span style="color:CadetBlue">Exploratory Data Analysis</span>
We start our journey by importing the libraries which are fundamental in order to work on the dataset, and the dataset itself.

In [1]:
# Import the libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import scipy.stats as stats
from scipy.stats import shapiro
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
%matplotlib tk
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

In [2]:
# Import the dataset
df = pd.read_csv('dataset.csv', index_col=0)

# Visualize the dataset
df

,label,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35
0,1,0.201113,-0.290211,-1.293273,-0.987445,0.375111,3.262836,-1.221763,-0.036009,1.435816,...,-1.220223,1.230786,2.145584,1.564664,0.584401,1.494508,0.287226,-0.852132,-0.759382,1.041190
1,2,1.628621,2.803231,-0.671938,0.328820,-0.074892,-6.121306,0.736208,-0.902092,-1.411483,...,-0.623831,2.929491,0.070468,-3.152713,-4.343053,0.909593,-1.577434,1.009611,-0.621972,-0.587301
2,0,1.100785,-1.583855,5.478388,-0.468925,-0.803414,-1.751841,-1.017715,-1.547837,-0.120226,...,-1.087988,-4.678151,-3.911785,-6.108772,-3.052242,-0.027595,-0.809724,3.607353,-0.021959,-2.211963
3,2,0.940043,-0.837257,-0.181058,-1.814802,1.129368,2.175010,-1.678725,1.800978,-0.716211,...,2.812262,2.075730,1.488613,-0.842421,-1.413351,0.405909,1.508635,-0.783645,-0.593335,-0.363084
4,2,-0.068997,-2.446645,2.682663,1.958176,0.013928,-0.289787,0.797141,0.106172,-0.531375,...,0.801879,-1.485274,1.815136,1.434652,-0.734058,0.779631,-1.585892,2.955094,-1.617037,0.730129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,-0.412984,-3.669425,2.356743,0.467361,0.110527,2.799420,1.007144,0.520349,-0.934891,...,1.261325,2.595839,2.017200,3.008557,1.816542,1.891215,1.422110,-1.145137,-0.016489,-0.584580
996,2,1.157384,-0.190413,1.928101,3.214276,0.997408,-1.338326,-0.267054,-0.815329,-1.114050,...,-0.371433,1.287553,2.317670,0.195089,0.148927,-0.251898,-1.338197,1.244734,1.060938,-0.544084
997,3,-0.419290,0.080442,-0.265915,1.015759,-0.338928,1.215722,-2.106663,-0.650065,-0.264707,...,2.219610,-0.724935,7.981604,3.304423,-3.472791,-0.034423,-0.780662,5.539539,-1.054519,4.222095
998,3,2.042390,-0.818479,2.949888,-0.625669,-0.218483,0.520263,-0.928446,0.134972,0.881317,...,-0.521867,0.524603,-1.532325,-3.241046,-0.836606,1.029299,-0.468110,0.059098,-0.744620,-1.327555


We search for missing values in the dataset, we do this by using the pandas DataFrame methods 'isna()' and 'sum()'.  
In particular:  
1. 'df.isna()' returns a DataFrame of the same shape as df, but with True at places where the original DataFrame has NaN (or missing) values, and False where it doesn't.    

2. 'df.isna().sum(): The sum() function when applied after isna() counts the number of True values along each column of the DataFrame returned by isna(). This effectively gives you the total number of missing values in each column.    

3. 'df.isna().sum().sum(): The second sum() function then adds up the sums of missing values from all columns, giving you the total number of missing values in the entire DataFrame.

In [3]:
total_missing_values = df.isna().sum().sum()
total_missing_values

0

There are no missing values in the dataset.

By taking a quick look at the dataset we soon notice that the first column shows the label of each datapoint. We therefore proceed to remove this column from the dataset and save it in a separate list, order to be able to process the features directly from the dataset.

In [4]:
label = df['label']
df_nolabel = df.drop('label', axis = 1)

Let's take a look at the distribution of the features:

In [5]:
df_nolabel.describe()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.282933,-0.134222,0.425977,0.104823,0.115679,0.413091,-0.039085,0.090042,0.054637,0.579426,...,0.133214,0.474120,0.686559,0.295006,-0.065978,0.154925,0.133279,0.297240,0.051504,0.307107
std,1.818610,1.960321,2.928722,1.966148,0.999951,2.594393,1.810147,1.004531,1.015530,3.410613,...,1.000638,1.850237,3.627684,3.218812,1.898731,1.009853,1.022389,2.179414,0.959773,1.828522
min,-6.767465,-8.091172,-8.976704,-5.834941,-2.869279,-7.531196,-6.957603,-2.943063,-2.846600,-12.305623,...,-2.882318,-5.431653,-10.873214,-12.435061,-5.892671,-2.966331,-2.886916,-6.412861,-3.257298,-6.405617
25%,-0.850264,-1.403214,-1.563170,-1.248238,-0.596544,-1.305197,-1.316516,-0.619754,-0.680124,-1.690841,...,-0.564726,-0.796360,-1.552170,-1.728238,-1.381113,-0.547456,-0.549123,-1.156720,-0.611668,-0.883565
50%,0.371561,-0.119651,0.490831,-0.012048,0.130476,0.171989,0.061171,0.067529,0.054134,0.630859,...,0.108615,0.541051,0.699022,0.439118,-0.065264,0.123897,0.139922,0.375740,0.068088,0.440117
75%,1.554514,1.147522,2.517349,1.396477,0.834385,1.991070,1.254006,0.754327,0.758453,3.054894,...,0.785315,1.741152,3.141299,2.487484,1.215040,0.875205,0.794936,1.879065,0.657521,1.571504
max,5.186449,6.958862,9.334812,6.197449,3.222567,8.363020,5.155523,3.343691,2.757545,11.341444,...,2.964663,6.420313,12.267814,9.117051,7.860133,2.867358,3.132727,6.167547,3.081773,6.903088


In order to become more familiar with the dataset, we plot some interesting graphs:

In [6]:
# Histograms
df.hist(figsize=(20,20), bins=20)
plt.subplots_adjust(hspace=0.5)
plt.show()

It can be observed that the occurrence of label 0 is slightly above 200 instances, along with labels 1, 2, and 3. Hence, the data indicates a relatively low level of variation among them.
From the histograms it seems that each feature is distributed pretty symmetrically, furthermore they resemble the shapes of some gaussians.

In [7]:
# Boxplots
df_nolabel.boxplot(figsize=(10,6))
plt.subplots_adjust(hspace=0.5)
plt.xticks(rotation=90) 
plt.show()

The boxplots seem to confirm our supposition, they are symmetrical and resemble those of some gaussians.
Let's check statistically if our supposition is true: first with some QQ-plots, then with some Shapiro tests.

In [8]:
# QQ-plots
num_rows = 5
num_cols = 7

fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 12))

for i, ax in enumerate(axes.flatten()):
    if i < df_nolabel.shape[1]:
        stats.probplot(df_nolabel.iloc[:, i], dist="norm", plot=ax)
        ax.set_xticks([])
        ax.set_yticks([])
    else:
        fig.delaxes(ax)

fig.tight_layout()

plt.show()

In [9]:
# Shapiro tests
count = 0
for i in range(df_nolabel.shape[1]):
    stat, p_value = shapiro(df_nolabel.iloc[:, i])
    alpha = 0.05
    if p_value > alpha:
        print("feature number {} is normally distributed".format(i+1))
        count = count + 1
    else:
        print("feature number {} is not normally distributed".format(i+1))
print("\n the number of features normally distributed is {}".format(count))


feature number 1 is not normally distributed
feature number 2 is normally distributed
feature number 3 is normally distributed
feature number 4 is normally distributed
feature number 5 is normally distributed
feature number 6 is not normally distributed
feature number 7 is not normally distributed
feature number 8 is normally distributed
feature number 9 is normally distributed
feature number 10 is normally distributed
feature number 11 is not normally distributed
feature number 12 is not normally distributed
feature number 13 is normally distributed
feature number 14 is normally distributed
feature number 15 is not normally distributed
feature number 16 is normally distributed
feature number 17 is normally distributed
feature number 18 is not normally distributed
feature number 19 is normally distributed
feature number 20 is normally distributed
feature number 21 is normally distributed
feature number 22 is normally distributed
feature number 23 is normally distributed
feature number 

The Shapiro test is very sensible and every small fluctuation could result in a false test, even if the data is normally distributed. In general we are satisfied with the result, every feature resembles a gaussian in the QQ-plots, some more than others.  

Let's take a look at the correlation.

In [10]:
# Correlations between features
corr = df_nolabel.corr()

# Heatmap
plt.figure(figsize=(14,14))
sns.heatmap(corr, annot=False, cmap='coolwarm')
plt.title('Correlations between features')
plt.show()

The features seem pretty independent between themselves. We could argue that the dataset is a sample from an independent multivariate normal distribution.

### <span style="color:CadetBlue">Normalization (and PCA visualization)</span>

We normalize our features. This is important because later on we will use K-NearestNeighbors classifier, that is very sensitive to scaled data. Random forests are an ensemble method built upon decision trees, that are indifferent to scaling, so we can keep the data normalized.

In the exploration of our dataset, we've leveraged boxplots as an analytical tool to visually examine the distributions of our features. From these visualizations, it's evident that the distributions display symmetry, which suggests a Gaussian trend. Outliers, represented by points beyond the 'whiskers' of the boxplot, are also present but they are symmetrically scattered and relatively close to the mean.

In terms of data preprocessing, this understanding of our data guides us towards the choice of a scaler. Given the observations, using the StandardScaler is a fitting choice for our dataset. The StandardScaler works effectively when the data is normally distributed and outliers are not extreme, as it standardizes features by removing the mean and scaling to unit variance. This technique helps preserve the essence of the original distributions, which aligns with our goal of capturing as much information as possible from the data.

On the other hand, the RobustScaler is more suitable for datasets with extreme outliers and non-symmetric distributions, as it scales features using statistics that are robust to outliers. In our case, however, the outliers are not extreme and the distributions are symmetric, which makes the StandardScaler a more appropriate option.

Hence, by choosing the StandardScaler, we're not only considering the nature of our data but also preserving crucial information embedded within our original distributions.

In [11]:
# Data scaling
scaler = StandardScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df_nolabel))
df_normalized

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,-0.045013,-0.079613,-0.587324,-0.555815,0.259575,1.098974,-0.653687,-0.125545,1.360738,0.312672,...,-1.353251,0.409161,0.402393,0.394647,0.342705,1.327176,0.150651,-0.527640,-0.845296,0.401664
1,0.740324,1.499205,-0.375066,0.113984,-0.190675,-2.519922,0.428518,-0.988153,-1.444422,0.272900,...,-0.756941,1.327721,-0.169915,-1.071651,-2.253723,0.747678,-1.674088,0.327027,-0.702056,-0.489387
2,0.449938,-0.739857,1.725988,-0.291959,-0.919597,-0.834884,-0.540906,-1.631307,-0.172275,-1.196385,...,-1.221034,-2.786048,-1.268204,-1.990480,-1.573555,-0.180830,-0.922814,1.519568,-0.076581,-1.378343
3,0.361506,-0.358812,-0.207373,-0.976827,1.014246,0.679465,-0.906258,1.704072,-0.759440,0.715088,...,2.678680,0.866057,0.221203,-0.353545,-0.709972,0.248659,1.345911,-0.496200,-0.672203,-0.366704
4,-0.193613,-1.180204,0.770921,0.943104,-0.101806,-0.271058,0.462197,0.016066,-0.577339,0.483573,...,0.668573,-1.059526,0.311257,0.354235,-0.352032,0.618919,-1.682365,1.220137,-1.739346,0.231463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-0.382856,-1.804282,0.659582,0.184482,-0.005154,0.920263,0.578270,0.428581,-0.974883,1.377909,...,1.127956,1.147302,0.366985,0.843451,0.991958,1.720209,1.261238,-0.662150,-0.070878,-0.487898
996,0.481075,-0.028678,0.513150,1.582287,0.882214,-0.675416,-0.126003,-0.901738,-1.151391,0.093618,...,-0.504578,0.439857,0.449854,-0.031057,0.113240,-0.403055,-1.439973,0.434964,1.052269,-0.465740
997,-0.386325,0.109560,-0.236362,0.463542,-0.454856,0.309526,-1.142788,-0.737137,-0.314618,0.223235,...,2.086109,-0.648379,2.011944,0.935414,-1.795155,-0.187595,-0.894374,2.406574,-1.152958,2.142139
998,0.967957,-0.349228,0.862210,-0.371721,-0.334345,0.041329,-0.491566,0.044750,0.814445,-0.164423,...,-0.654992,0.027298,-0.611959,-1.099108,-0.406068,0.866275,-0.588514,-0.109324,-0.829908,-0.894427


Just in order to visualize the distribution of the labels geometrically, we perform a PCA.

In [12]:
# 3-components PCA
pca = PCA(n_components = 3)
principal_components = pca.fit_transform(df_normalized)
df_pca = pd.DataFrame(principal_components, columns=['PC1', 'PC2', 'PC3'])

In [13]:
# Visualizing the PCA
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(df_pca['PC1'], df_pca['PC2'], df_pca['PC3'], c=df['label'])
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.set_title('PCA: PC1 vs PC2 vs PC3 (Color by Label)')
fig.colorbar(scatter, label='Label')
plt.show()

The plot seems to be messy, it could be due to the fact that we project our datapoints onto a 3-dimensional space, ignoring much information.
It could also be due to the fact that our dataset is intrinsically messy, anyway we will keep all the dimensions when training the classifiers since neither should show big computational problems with this small dataset.

### <span style="color:CadetBlue">Models Training</span>
This is the most important part of the notebook. We have decided to divide it in three parts:  
<span style="color:CadetBlue">1. </span> Firstly we will train both models without any hyperparameter tuning.  
<span style="color:CadetBlue">2. </span> Then, we will train both models using hyperparameters tuning, realized by GridSearch cross-validation on a small range of possibilities in order to keep the notebook computationally feasible, and keeping all the features.  
<span style="color:CadetBlue">3. </span> In the end, we will again train both models using hyperparameters tuning, realized by RandomizedSearch on a bigger range of possibilities, this time including the number of features as a hyperparameter.

It is wise to split the dataset in X_train and X_test (as well as the labels), so that we will train our classifiers on X_train, and evaluate their performance on unseen data.

In [14]:
# Splitting
X_train, X_test, y_train, y_test = train_test_split(df_normalized, label, test_size = 0.3, random_state = 42)

##### <span style="color:CadetBlue">1. No Hyperparameter Tuning</span>
For the time being, we just train the two classifiers without hyperparameter tuning.

In [15]:
# K-NN without tuning
knn1 = KNeighborsClassifier()
knn1.fit(X_train, y_train)

y_pred = knn1.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.48


In [16]:
# Random Forest without tuning
random_forest1 = RandomForestClassifier()
random_forest1.fit(X_train, y_train)

y_pred = random_forest1.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.6233333333333333


The first part shows that without any hyperparameter tuning the Random Forest seems to work better. It is remarkable that if we train them again (with the same seed in train_test_split), the k-NearestNeighbors will always return the same accuracy, because it isn't stochastic. Instead Random Forest will show different results for each run.

##### <span style="color:CadetBlue">2. Simple Hyperparameter Tuning</span>
No we will tune the hyperparameters with GridSearch (without including the number of features in the hyperparameters).

GridSearch searches through all the possible choices of hyperparameters, therefore we keep the number of choices small, to keep the notebook computationally feasible.

The hyperparameters involved in the optimization of k-NearestNeighbors are:  
1. The number of labeled neighbors considered when deciding which label to assign to a new datapoint.
2. The weights, which refer to the method used to weigh or account for the contribution of the neighbors when making predictions. In particular, in 'uniform' all points in each neighborhood are weighted equally. This means that each of the 'k' nearest neighbors has an equal vote in determining the class of the object in question. Instead in 'distance' weights are assigned to the neighbors based on the inverse of their distance. Closer neighbors of a query point will have a greater influence than neighbors which are further away.  
3. The metric, that is to say the method we use to calculate the distance between datapoints.

In [17]:
# KNN simple tuning

# Creating the classifier
knn2 = KNeighborsClassifier()

# Defining the hyperparameters
knn2_parameters = {
    "n_neighbors": [3, 5, 7, 9],
    "weights": ["uniform", "distance"],
    "metric": ["euclidean", "manhattan"]
}

# GridSearch and fitting
knn2_clf = GridSearchCV(knn1, knn2_parameters, cv=5)
knn2_clf.fit(X_train, y_train)

print(f"Best parameters for k-NN: {knn2_clf.best_params_}")

# Prediction
knn2_y_pred = knn2_clf.predict(X_test)

# Accuracy
print(f"Accuracy for k-NN: {accuracy_score(y_test, knn2_y_pred)}")

Best parameters for k-NN: {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'distance'}
Accuracy for k-NN: 0.59


We notice an improvement.

Moving to the Random Forest, the hyperparameters involved in the optimization are:  
1. The number of estimators, which controls the number of trees in the random forest. Each tree provides a prediction, and the class with the most votes becomes the model's prediction.  
2. The max_depth, this parameter represents the depth of each tree in the forest. The deeper the tree, the more splits it has, and it captures more information about the data. However, deeper trees are more complex and can lead to overfitting. (If max_depth = None, nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples).
3. The min_samples_split represents the minimum number of samples required to split an internal node.

In [18]:
# Random Forest simple tuning

# Creating the classifier
random_forest2 = RandomForestClassifier()

# Defining the hyperparameters
random_forest2_parameters = {
    "n_estimators": [10, 50, 100, 200],
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10]
}

# GridSearch and fitting
random_forest2_clf = GridSearchCV(random_forest1, random_forest2_parameters, cv=5)
random_forest2_clf.fit(X_train, y_train)

print(f"Best parameters for Random Forest: {random_forest2_clf.best_params_}")

# Prediction
random_forest2_y_pred = random_forest2_clf.predict(X_test)

# Accuracy
print(f"Accuracy for Random Forest: {accuracy_score(y_test, random_forest2_y_pred)}")


Best parameters for Random Forest: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 200}
Accuracy for Random Forest: 0.6566666666666666


The Random Forest still shows a better performance.

##### <span style="color:CadetBlue">3. Advanced Tuning</span>
In order to try to improve our accuracy, we can include the number of features in the hyperparameters.  
Having a simpler model (i.e. less features) could help to prevent overfitting, resulting in a higher accuracy. Unfortunately, the number of possible subsets of features is exponential in the number of features, so we cannot compute them by hand with a GridSearch, and neither a RandomizedSearch would work well because we would try only a small family of all the possible subsets, so we have to think of another trick.  
We create a gerarchy of importance among the features: we can do it thanks to feature_importance_ method implemented in RandomForest.  

In [19]:
# Creating a gerarchy among the features
importances = random_forest1.feature_importances_

feature_importances = pd.DataFrame(random_forest1.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)

feature_importances.index = feature_importances.index + 1

plt.figure(figsize=(10, 10))
plt.title('Feature Importances')
plt.barh(feature_importances.index, feature_importances['importance'], color='CadetBlue', align='center')
plt.xlabel('Relative Importance')
plt.show()
print(feature_importances)

    importance
15    0.055997
10    0.048311
27    0.046381
6     0.046024
30    0.042589
2     0.041187
33    0.039565
3     0.038801
1     0.038776
7     0.036338
28    0.034627
12    0.034595
35    0.034283
4     0.034146
11    0.033566
29    0.031052
25    0.029261
9     0.020967
17    0.019812
18    0.019247
34    0.019204
31    0.018917
13    0.018735
16    0.018705
22    0.018525
24    0.018468
5     0.018458
19    0.018454
32    0.018413
23    0.018408
21    0.018219
8     0.018085
20    0.017702
26    0.017256
14    0.016928


In the table above, it is shows that the 15th feature is the most important, the10th feature is the second most important and so on...  
We save this gerarchy into a list and move the indices back of one unit (we do it only because of technical issues: most methods in python starts counting from 0).

In [20]:
important_features = feature_importances.index.tolist()
important_features = [feat - 1 for feat in important_features]

Let's train our classifiers:

Now we use RandomizedSearch cross-validation, so we can choose bigger ranges for our hyperparameters, since the method itself guarantees that the training will remain computationally feasible. In particular, we made the number of possible n_neighbors choices bigger, we introduced two metrics, and a hyperparameter 'p', which only regards the Minkowski metric (it chooses its polynomial degree).

In [21]:
# KNN advanced tuning
knn3_parameters = {
    "n_neighbors": range(1, 50),
    "weights": ["uniform", "distance"],
    "metric": ["euclidean", "manhattan", "chebyshev", "minkowski"],
    "p": [1, 2, 3, 4, 5]
}

# List to store the best accuracy for each set of features
knn3_best_accuracies = []

# Number of features to consider in each iteration
feature_counts = np.arange(5, len(important_features) + 1, 5)

knn3_best_accuracy = 0
knn3_best_num_features = 0

# Loop over the number of features to consider
for num_features in feature_counts:

    # Select the features
    selected_features = important_features[:num_features]
    print([feature + 1 for feature in important_features[:num_features]])
    X_train_selected = X_train[selected_features]
    X_test_selected = X_test[selected_features]

    # Create and fit the model
    knn3 = KNeighborsClassifier()
    clf = RandomizedSearchCV(knn3, knn3_parameters, cv=5)
    clf.fit(X_train_selected, y_train)

    # Predict the test set results
    y_pred = clf.predict(X_test_selected)

    # Compute the accuracy
    accuracy = accuracy_score(y_test, y_pred)
    knn3_best_accuracies.append(accuracy)

    # Check if this model is better than the previous best
    if accuracy > knn3_best_accuracy:
        knn3_best_accuracy = accuracy
        knn3_best_num_features = num_features
        knn3_final = clf

    print(f"Best parameters for {num_features} features: {clf.best_params_}")
    print(f"Best accuracy for {num_features} features: {accuracy}\n")

# Plot the accuracies
plt.figure(figsize=(10, 7))
plt.plot(feature_counts, knn3_best_accuracies, marker='o')
plt.title('Best k-NN Accuracy vs. Number of Features')
plt.xlabel('Number of Features')
plt.ylabel('Best Accuracy')
plt.grid(True)
plt.show()

[15, 10, 27, 6, 30]
Best parameters for 5 features: {'weights': 'distance', 'p': 5, 'n_neighbors': 32, 'metric': 'minkowski'}
Best accuracy for 5 features: 0.54

[15, 10, 27, 6, 30, 2, 33, 3, 1, 7]
Best parameters for 10 features: {'weights': 'distance', 'p': 4, 'n_neighbors': 2, 'metric': 'euclidean'}
Best accuracy for 10 features: 0.67

[15, 10, 27, 6, 30, 2, 33, 3, 1, 7, 28, 12, 35, 4, 11]
Best parameters for 15 features: {'weights': 'uniform', 'p': 2, 'n_neighbors': 9, 'metric': 'minkowski'}
Best accuracy for 15 features: 0.6966666666666667

[15, 10, 27, 6, 30, 2, 33, 3, 1, 7, 28, 12, 35, 4, 11, 29, 25, 9, 17, 18]
Best parameters for 20 features: {'weights': 'distance', 'p': 4, 'n_neighbors': 7, 'metric': 'manhattan'}
Best accuracy for 20 features: 0.6866666666666666

[15, 10, 27, 6, 30, 2, 33, 3, 1, 7, 28, 12, 35, 4, 11, 29, 25, 9, 17, 18, 34, 31, 13, 16, 22]
Best parameters for 25 features: {'weights': 'distance', 'p': 2, 'n_neighbors': 7, 'metric': 'manhattan'}
Best accuracy for

In [22]:
print(f"Best model parameters: {knn3_final.best_params_}")
print(f"Number of features in the best model: {knn3_best_num_features}")
print(f"Best accuracy: {knn3_best_accuracy}")

Best model parameters: {'weights': 'uniform', 'p': 2, 'n_neighbors': 9, 'metric': 'minkowski'}
Number of features in the best model: 15
Best accuracy: 0.6966666666666667


Now we increase the range of our hyperparameters for Random Forest.  
We added min_samples_leaf, which governs the minimum number of samples required to be at a leaf node, and we added max_features, which represents the number of features to consider when looking for the best split:  
if 'sqrt': then max_features = sqrt(n_features),  
if 'log2': Then max_features = log2(n_features).

In [23]:
# Random Forest advanced tuning
random_forest3_parameters = {
    "n_estimators": [10, 50, 100, 200, 300, 400, 500],  
    "max_depth": [None, 10, 20, 30, 40, 50],
    "min_samples_split": [2, 5, 10, 15, 20],
    "min_samples_leaf": [1, 2, 4, 6, 8, 10],
    "max_features": ['sqrt', 'log2']
}
# List to store the best accuracy for each set of features
best_accuracies = []

# Number of features to consider in each iteration
feature_counts = np.arange(5, len(important_features) + 1, 5)

random_forest3_best_accuracy = 0
random_forest3_best_num_features = 0

# Loop over the number of features to consider
for num_features in feature_counts:

    # Select the features
    selected_features = important_features[:num_features]
    print([feature + 1 for feature in important_features[:num_features]])
    X_train_selected = X_train[selected_features]
    X_test_selected = X_test[selected_features]

    # Create and fit the model
    random_forest3 = RandomForestClassifier()
    clf = RandomizedSearchCV(random_forest3, random_forest3_parameters, cv=5)
    clf.fit(X_train_selected, y_train)

    # Predict the test set results
    y_pred = clf.predict(X_test_selected)

    # Compute the accuracy
    accuracy = accuracy_score(y_test, y_pred)
    best_accuracies.append(accuracy)

    # Check if this model is better than the previous best
    if accuracy > random_forest3_best_accuracy:
        random_forest3_best_accuracy = accuracy
        random_forest3_best_num_features = num_features
        random_forest3_final = clf

    print(f"Best parameters for {num_features} features: {clf.best_params_}")
    print(f"Best accuracy for {num_features} features: {accuracy}\n")

# Plot the accuracies
plt.figure(figsize=(10, 7))
plt.plot(feature_counts, best_accuracies, marker='o')
plt.title('Best k-NN Accuracy vs. Number of Features')
plt.xlabel('Number of Features')
plt.ylabel('Best Accuracy')
plt.grid(True)
plt.show()

[15, 10, 27, 6, 30]
Best parameters for 5 features: {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': None}
Best accuracy for 5 features: 0.5633333333333334

[15, 10, 27, 6, 30, 2, 33, 3, 1, 7]
Best parameters for 10 features: {'n_estimators': 400, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 40}
Best accuracy for 10 features: 0.6433333333333333

[15, 10, 27, 6, 30, 2, 33, 3, 1, 7, 28, 12, 35, 4, 11]
Best parameters for 15 features: {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 10}
Best accuracy for 15 features: 0.6766666666666666

[15, 10, 27, 6, 30, 2, 33, 3, 1, 7, 28, 12, 35, 4, 11, 29, 25, 9, 17, 18]
Best parameters for 20 features: {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': None}
Best accuracy for 20 features: 0.69

[15, 10, 27, 6, 30, 2, 33, 3, 1, 7, 28, 12, 35, 4, 

In [24]:
print(f"Best model parameters: {random_forest3_final.best_params_}")
print(f"Number of features in the best model: {random_forest3_best_num_features}")
print(f"Best accuracy: {random_forest3_best_accuracy}")

Best model parameters: {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': None}
Number of features in the best model: 20
Best accuracy: 0.69


### <span style="color:CadetBlue">Conclusions and Final Prediction</span>


In the initial round of model fitting, where no hyperparameter tuning was conducted, Random Forest performed better than KNN. The performance comparison remained the same in the second stage of testing, which involved hyperparameter tuning but did not consider the number of features as a tunable hyperparameter. This can be attributed to the inherent strength of Random Forest, a robust ensemble method that leverages a multitude of decision trees to make predictions. This ensemble approach reduces overfitting and increases the model's generalization ability, thereby leading to improved performance.

However, the scenario changed when feature importance was incorporated into the model fitting. Random Forest was used to generate a ranking of feature importance, which was then included as an additional hyperparameter in the tuning phase. With this information, KNN performed slightly better than the Random Forest. The reason for this could be that feature importance provides a form of feature selection. When the most relevant features were emphasized and introduced into the hyperparameter space, the KNN algorithm's performance improved. By focusing on a smaller, more relevant set of features, KNN might have been able to reduce noise in the data and make more accurate predictions.

In conclusion, both Random Forest and KNN have demonstrated their strengths in different aspects of this experiment. While Random Forest generally performs better with a larger feature set, KNN seems to excel when feature selection is taken into account. Therefore, it's crucial to remember that the choice of the classifier and its hyperparameters should be informed by the specific characteristics of the data at hand, rather than relying on the assumption that one classifier will universally outperform the other.  

It is important to say that given the closeness of the performances of the classifier, and the stochastic nature of Random Forest, the result also depends slightly on the random seed.

In [25]:
# Import the test dataset
df_test = pd.read_csv('test_data.csv', index_col=0)

# Visualize the dataset
df_test

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35
id,,,,,,,,,,,,,,,,,,,,,
0,-2.259876,1.259865,-0.908350,-1.853991,0.385436,-1.105350,1.991604,-0.975180,-0.610046,-2.077745,...,0.582761,-0.758695,-1.753105,0.902077,0.079239,0.731773,2.401233,-0.056351,-1.691496,0.692310
1,-2.529047,1.120999,-4.326973,-0.893663,0.598676,-0.352028,-1.222735,-0.078255,-0.574625,0.527005,...,-0.304697,-0.329206,1.524611,0.662350,-0.898723,2.235237,0.596613,-0.914091,-0.821066,-1.109875
2,0.012222,-0.177944,-1.297251,-0.147131,-1.011613,1.365393,-1.620815,-0.499335,1.425400,2.320669,...,0.359054,1.573846,2.876398,0.634198,-0.699412,-0.424788,0.146164,-0.335836,-0.473143,-0.192600
3,-0.089448,-2.826220,3.938348,-1.599376,1.190689,2.446367,1.675493,-0.457380,0.497983,1.533945,...,1.337707,-1.240190,0.553714,1.999368,-0.796620,1.076099,0.907682,2.919312,-0.316218,2.976172
4,0.318715,2.223043,-1.566211,1.067817,1.312906,-1.992862,1.511649,0.590657,-0.708409,-3.091795,...,-0.455276,-1.605761,-3.269420,-0.527665,1.735746,0.167339,0.612655,-0.788140,-0.006253,0.208321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3.541143,-0.192635,0.243902,1.083794,1.907259,3.304881,-0.295063,-0.435150,2.418466,-0.104594,...,-0.263075,-0.052187,-2.089612,-0.257640,3.437469,-1.640569,-0.345324,-1.990600,1.105713,1.017189
996,0.791110,1.798410,-1.411891,-0.161851,3.359425,2.244013,-1.182066,-1.770667,-0.640782,-3.740578,...,-1.021918,-1.518739,-0.928221,-0.176392,1.954623,-1.229088,1.287181,-0.600885,-1.487602,1.171236
997,0.473525,-0.185459,0.951580,0.334926,-1.087654,-0.260079,-2.475909,0.493006,0.840025,-0.238076,...,0.666892,0.124335,0.803261,-2.884849,-1.499423,-0.120510,0.685016,0.393198,-0.495217,-2.488886


In [26]:
# Normalize the test dataset
scaler = StandardScaler()
df_test_normalized = pd.DataFrame(scaler.fit_transform(df_test))
df_test_normalized

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,-1.434444,0.657116,-0.487240,-1.030707,0.265214,-0.538569,1.097759,-1.135797,-0.730989,-0.734794,...,0.448822,-0.635366,-0.594249,0.264870,0.088284,0.614043,2.414138,-0.180984,-1.785150,0.208651
1,-1.578435,0.585199,-1.650163,-0.523867,0.477534,-0.240718,-0.688568,-0.200187,-0.695890,0.062179,...,-0.439690,-0.398324,0.309225,0.189487,-0.430704,2.038327,0.558815,-0.589524,-0.902993,-0.776633
2,-0.219003,-0.087507,-0.619533,-0.129864,-1.125807,0.438322,-0.909796,-0.639429,1.285976,0.610984,...,0.224849,0.652005,0.681833,0.180634,-0.324933,-0.481608,0.095710,-0.314102,-0.550384,-0.275144
3,-0.273390,-1.459017,1.161476,-0.896326,1.066993,0.865722,0.922084,-0.595664,0.366979,0.370271,...,1.204664,-0.901112,0.041606,0.609921,-0.376520,0.940235,0.878624,1.236319,-0.391345,1.457275
4,-0.055046,1.155935,-0.711026,0.511358,1.188682,-0.889477,0.831030,0.497575,-0.828460,-1.045061,...,-0.590447,-1.102877,-1.012207,-0.184722,0.967363,0.079335,0.575307,-0.529534,-0.077204,-0.055954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.668766,-0.095116,-0.095275,0.519791,1.780471,1.205164,-0.173027,-0.572476,2.270025,-0.131071,...,-0.398018,-0.245432,-0.687004,-0.099811,1.870437,-1.633360,-0.409587,-1.102263,1.049741,0.386268
996,0.197657,0.936022,-0.658531,-0.137633,3.226370,0.785714,-0.665967,-1.965594,-0.761447,-1.243569,...,-1.157761,-1.054848,-0.366876,-0.074262,1.083517,-1.243550,1.268785,-0.440344,-1.578509,0.470487
997,0.027768,-0.091399,0.145458,0.124555,-1.201519,-0.204363,-1.385003,0.395713,0.705916,-0.171912,...,0.533053,-0.148006,0.110391,-0.925955,-0.749485,-0.193354,0.649702,0.033136,-0.572756,-1.530560
998,-0.773249,-0.814713,-0.988576,1.720621,0.813878,-0.584251,1.603908,0.047005,-0.253878,2.029927,...,-0.226124,1.761491,0.359438,1.469598,1.741266,0.730020,0.598912,-1.703245,-1.226880,-0.882881


In [27]:
# Select the best features from df_test
df_test_normalized_selected = df_test_normalized[important_features[:knn3_best_num_features]]

# Use the best model to predict the test set results
df_test_predictions = knn3_final.predict(df_test_normalized_selected)
df_test_predictions

array([1, 1, 1, 3, 1, 3, 2, 3, 1, 3, 2, 0, 3, 0, 0, 2, 2, 3, 0, 2, 2, 3,
       0, 0, 1, 1, 2, 2, 0, 3, 1, 2, 1, 3, 2, 2, 2, 3, 0, 3, 0, 1, 3, 2,
       0, 3, 1, 1, 3, 3, 2, 2, 1, 2, 3, 1, 0, 1, 2, 2, 3, 1, 2, 1, 2, 3,
       3, 2, 0, 2, 1, 3, 2, 1, 1, 0, 0, 2, 3, 1, 0, 0, 3, 1, 1, 0, 1, 0,
       0, 2, 3, 1, 1, 0, 1, 3, 3, 3, 0, 3, 0, 1, 0, 0, 2, 3, 3, 3, 2, 0,
       2, 1, 1, 2, 3, 2, 2, 1, 0, 0, 3, 2, 3, 2, 0, 2, 0, 0, 1, 2, 2, 3,
       3, 2, 3, 3, 3, 0, 1, 2, 0, 0, 3, 0, 2, 2, 1, 3, 2, 0, 2, 2, 1, 3,
       0, 0, 3, 0, 0, 3, 1, 1, 0, 2, 2, 1, 1, 3, 0, 2, 2, 0, 1, 0, 2, 2,
       2, 1, 1, 2, 3, 0, 1, 2, 2, 3, 2, 1, 2, 2, 1, 3, 1, 0, 0, 1, 2, 2,
       3, 2, 2, 1, 1, 3, 1, 2, 0, 1, 2, 2, 3, 2, 1, 2, 3, 1, 1, 3, 1, 2,
       1, 2, 1, 3, 1, 1, 0, 2, 1, 3, 2, 0, 3, 2, 3, 0, 0, 3, 2, 0, 2, 2,
       1, 2, 1, 0, 1, 0, 1, 3, 0, 2, 2, 3, 3, 1, 0, 2, 0, 3, 3, 2, 0, 2,
       0, 1, 3, 1, 0, 1, 1, 2, 2, 1, 3, 3, 2, 0, 0, 3, 3, 3, 0, 1, 2, 2,
       0, 0, 1, 1, 2, 1, 0, 0, 0, 2, 2, 2, 2, 3, 3,

In [29]:
df_test_predictions = np.array(df_test_predictions)

# Save to a txt file
np.savetxt('test_predictions.txt', df_test_predictions, fmt='%d', newline='\n')

### <span style="color:CadetBlue">Extra: K-NN and PCA visualization</span>
Just for the sake of curiosity we take the predictions on the labelled test dataset, and then we graphically visualize the mislabelled points after performing a PCA.

In [30]:
# Select the best number of features
selected_features = important_features[:knn3_best_num_features]

# Use the best KNN model to make predictions on the test set
knn3_test_predictions = knn3_final.predict(X_test[selected_features])

# Perform PCA on the test data
X_test_pca = pca.fit_transform(X_test[selected_features])

# Create a boolean array that indicates whether each prediction was correct
correct_predictions = (knn3_test_predictions == y_test)

# Plot the PCA-transformed test data
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(X_test_pca[:, 0], X_test_pca[:, 1], X_test_pca[:, 2], 
                     c=['green' if correct else 'red' for correct in correct_predictions])

# Create a legend for the colors
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper right", title="Predictions")
ax.add_artist(legend1)

# Add labels
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_zlabel('Principal Component 3')
plt.title('PCA of Test Data (Green = Correct Prediction, Red = Incorrect Prediction)')
plt.show()

It seems to be that the errors are randomly distributed, this could be due to the fact the we lost information in the PCA, but we could argue that the dataset itself wasn't very suited for this kind of classifiers.    
We are satisfied with our work, we managed to get almost 70% accuracy on the labelled test dataset, which we believe is satisfactory.